# Inventory Optimization for Retail - Data Preprocessing

This notebook focuses on loading, cleaning, and preprocessing the retail sales and inventory data for our predictive analytics model.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Set plotting style
plt.style.use('seaborn-whitegrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Data Loading

First, we need to load our data from the CSV/Excel files. We'll check for the presence of these files in the data directory.

In [ ]:
# List files in the data directory
data_dir = '../data/'
files = os.listdir(data_dir)
print(f"Files in data directory: {files}")

# Function to load data based on file extension
def load_data(file_path):
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    elif file_path.endswith(('.xlsx', '.xls')):
        return pd.read_excel(file_path)
    else:
        print(f"Unsupported file format: {file_path}")
        return None

## 2. Data Exploration

Let's explore the dataset to understand its structure, check for missing values, and get a sense of the data distribution.

In [ ]:
# Load the data (update file paths as needed)
# Example: sales_data = load_data(os.path.join(data_dir, 'sales_data.csv'))

# For demonstration, let's create a sample dataset
# This will be replaced with actual data loading when files are available

# Sample sales data
dates = pd.date_range(start='2022-01-01', end='2023-12-31', freq='D')
product_ids = [f'P{i:03d}' for i in range(1, 11)]
store_ids = [f'S{i:02d}' for i in range(1, 6)]

# Create sample data
np.random.seed(42)
sales_data = []

for date in dates:
    for product_id in product_ids:
        for store_id in store_ids:
            # Base sales with seasonal pattern
            base_sales = 50 + 30 * np.sin(2 * np.pi * date.dayofyear / 365)
            
            # Add product-specific variation
            product_factor = int(product_id[1:]) / 10
            
            # Add store-specific variation
            store_factor = int(store_id[1:]) / 5
            
            # Add random noise
            noise = np.random.normal(0, 10)
            
            # Calculate sales quantity
            sales_qty = max(0, int(base_sales * product_factor * store_factor + noise))
            
            # Calculate inventory level (simple formula for demonstration)
            inventory_level = max(0, int(sales_qty * 1.5 + np.random.normal(0, 20)))
            
            sales_data.append({
                'Date': date,
                'Product_ID': product_id,
                'Store_ID': store_id,
                'Sales_Quantity': sales_qty,
                'Inventory_Level': inventory_level
            })

# Convert to DataFrame
df_sales = pd.DataFrame(sales_data)

# Display the first few rows
df_sales.head()

In [ ]:
# Basic data exploration
print(f"Dataset shape: {df_sales.shape}")
print("\nData types:")
print(df_sales.dtypes)
print("\nSummary statistics:")
print(df_sales.describe())
print("\nMissing values:")
print(df_sales.isnull().sum())

## 3. Data Visualization

Let's visualize the data to better understand patterns and relationships.

In [ ]:
# Aggregate sales by date
daily_sales = df_sales.groupby('Date')['Sales_Quantity'].sum().reset_index()

# Plot daily sales
plt.figure(figsize=(15, 6))
plt.plot(daily_sales['Date'], daily_sales['Sales_Quantity'])
plt.title('Daily Total Sales')
plt.xlabel('Date')
plt.ylabel('Sales Quantity')
plt.grid(True)
plt.tight_layout()
plt.savefig('../images/daily_sales.png')
plt.show()

In [ ]:
# Aggregate sales by product
product_sales = df_sales.groupby('Product_ID')['Sales_Quantity'].sum().sort_values(ascending=False).reset_index()

# Plot product sales
plt.figure(figsize=(12, 6))
sns.barplot(x='Product_ID', y='Sales_Quantity', data=product_sales)
plt.title('Total Sales by Product')
plt.xlabel('Product ID')
plt.ylabel('Sales Quantity')
plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig('../images/product_sales.png')
plt.show()

In [ ]:
# Aggregate sales by store
store_sales = df_sales.groupby('Store_ID')['Sales_Quantity'].sum().sort_values(ascending=False).reset_index()

# Plot store sales
plt.figure(figsize=(10, 6))
sns.barplot(x='Store_ID', y='Sales_Quantity', data=store_sales)
plt.title('Total Sales by Store')
plt.xlabel('Store ID')
plt.ylabel('Sales Quantity')
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig('../images/store_sales.png')
plt.show()

In [ ]:
# Sales vs Inventory scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Sales_Quantity', y='Inventory_Level', data=df_sales.sample(1000), alpha=0.6)
plt.title('Sales Quantity vs Inventory Level')
plt.xlabel('Sales Quantity')
plt.ylabel('Inventory Level')
plt.grid(True)
plt.tight_layout()
plt.savefig('../images/sales_vs_inventory.png')
plt.show()

## 4. Data Preprocessing

Now, let's preprocess the data for our time series forecasting model.

In [ ]:
# Ensure Date is in datetime format
df_sales['Date'] = pd.to_datetime(df_sales['Date'])

# Extract date features
df_sales['Year'] = df_sales['Date'].dt.year
df_sales['Month'] = df_sales['Date'].dt.month
df_sales['Day'] = df_sales['Date'].dt.day
df_sales['DayOfWeek'] = df_sales['Date'].dt.dayofweek
df_sales['Quarter'] = df_sales['Date'].dt.quarter
df_sales['WeekOfYear'] = df_sales['Date'].dt.isocalendar().week

# Create a flag for weekends
df_sales['IsWeekend'] = df_sales['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# Display the enhanced dataset
df_sales.head()

In [ ]:
# Aggregate data to weekly level for time series forecasting
df_weekly = df_sales.groupby(['Year', 'WeekOfYear', 'Product_ID', 'Store_ID'])[
    'Sales_Quantity', 'Inventory_Level'
].agg({
    'Sales_Quantity': 'sum',
    'Inventory_Level': 'mean'
}).reset_index()

# Create a date column for the week
df_weekly['Week_Start'] = df_weekly.apply(
    lambda row: pd.to_datetime(f"{row['Year']}-W{row['WeekOfYear']:02d}-1", format='%Y-W%W-%w'),
    axis=1
)

# Display the weekly aggregated data
df_weekly.head()

In [ ]:
# Create lag features for each product-store combination
def create_lag_features(group, lags=[1, 2, 3, 4]):
    for lag in lags:
        group[f'Sales_Lag_{lag}'] = group['Sales_Quantity'].shift(lag)
    return group

# Apply the function to each product-store group
df_weekly_with_lags = df_weekly.sort_values(['Product_ID', 'Store_ID', 'Week_Start']).groupby(['Product_ID', 'Store_ID']).apply(create_lag_features).reset_index(drop=True)

# Create rolling mean features
def create_rolling_features(group, windows=[2, 4, 8]):
    for window in windows:
        group[f'Sales_Rolling_{window}'] = group['Sales_Quantity'].shift(1).rolling(window=window, min_periods=1).mean()
    return group

# Apply the function to each product-store group
df_weekly_features = df_weekly_with_lags.sort_values(['Product_ID', 'Store_ID', 'Week_Start']).groupby(['Product_ID', 'Store_ID']).apply(create_rolling_features).reset_index(drop=True)

# Drop rows with NaN values (first few weeks for each product-store combination)
df_weekly_features = df_weekly_features.dropna()

# Display the final dataset with features
df_weekly_features.head()

In [ ]:
# Save the processed data
df_weekly_features.to_csv('../data/processed_weekly_data.csv', index=False)
print("Processed data saved to '../data/processed_weekly_data.csv'")

## 5. Data Splitting

Split the data into training and testing sets for model evaluation.

In [ ]:
# Sort by date
df_weekly_features = df_weekly_features.sort_values('Week_Start')

# Determine the split point (e.g., use the last 8 weeks for testing)
split_date = df_weekly_features['Week_Start'].max() - pd.Timedelta(weeks=8)

# Split the data
train_data = df_weekly_features[df_weekly_features['Week_Start'] <= split_date]
test_data = df_weekly_features[df_weekly_features['Week_Start'] > split_date]

print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")

# Save the train and test datasets
train_data.to_csv('../data/train_data.csv', index=False)
test_data.to_csv('../data/test_data.csv', index=False)
print("Train and test data saved to '../data/'")

## 6. Summary

In this notebook, we have:
1. Loaded and explored the sales and inventory data
2. Visualized key patterns and relationships
3. Preprocessed the data for time series forecasting
4. Created lag and rolling features
5. Split the data into training and testing sets

The processed data is now ready for model development in the next notebook.